In [2]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import webbrowser
import time
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from git_ignore.config import *
import plotly.graph_objects as go

In [4]:
auth = tweepy.OAuthHandler(twitter_consumer_key, twitter_consumer_secret, twitter_callback_uri)
redirect_url = auth.get_authorization_url()

In [5]:
# auth.get_access_token(user_pin_input)
api = tweepy.API(auth)


# Get popular/recent tweets from keyword search

In [6]:
tweets = tweepy.Cursor(api.search, q='Drake', result_type='popular').items(10)

keyword_df = pd.DataFrame()
name = []
location = []
text = []

for tweet in tweets:
    name.append(tweet.user.name)
    location.append(tweet.user.location)
    text.append(tweet.text)

keyword_df['name'] = name
keyword_df['location'] = location
keyword_df['text'] = text

keyword_df

,name,location,text
0,Mrs. Petty,Queendom,Thank you so much for fkn w|me on live yall. #...
1,$Oxtail is a hustle,Toronto,New Drake Nicki and Wayne on the same night as...
2,Complex Music,,YMCMB BACK.\n\nNicki announces a new collab wi...
3,concrete boy boat,,Drake so fye lol
4,Sizwe Dhlomo,,"Nicki, Drake &amp; Wayne... It’s feeling like ..."
5,TIDAL,,"For the first time ever, @NICKIMINAJ's classic..."
6,XXL Magazine,,"“Seeing Green” by Nicki Minaj, Drake and Lil W..."
7,Mula,,THE BIG 3 🤑🤑🤑 TRIPLE GOATED 🐐🐐🐐 #BEAMMEUPSCOTT...
8,Mula,,SEEING GREEN OUT NOW 🤑🔥🐐\n\n@NICKIMINAJ ft. @L...
9,†,,"Wayne,Nicki,Drake \n\nthey understood the assi..."


In [7]:


fig = go.Figure()

fig.add_trace(go.Table(
        header=dict(values=list(keyword_df.columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[keyword_df.name, keyword_df.location, keyword_df.text],
                   fill_color='lavender',
                   align='left'))
    )


fig.show()

ModuleNotFoundError: No module named 'plotly'

# Get tweets from keyword search

# Get location heatmap using location woeid

# Trending tweets from location keyword

In [15]:
cwoeid = 23424775
woeid = 1
  
# fetching the trends
rj_trends = api.trends_place(id = cwoeid)
trends = []
for trend in rj_trends[0]['trends']: 
    if trend['tweet_volume'] is not None and trend['tweet_volume'] > 10000: 
        trends.append((trend['name'], trend['tweet_volume']))

trends.sort(key=lambda x:-x[1])
trends
# rj_trends

[('Nicki', 1145063),
 ('COVID-19', 697243),
 ('The CDC', 544442),
 ('J Cole', 400579),
 ('olivia rodrigo', 171481),
 ('Seeing Green', 138752),
 ('Itty Bitty Piggy', 122772),
 ('Marjorie Taylor Greene', 111985),
 ('Beam Me Up Scotty', 75506),
 ('Toronto', 35492),
 ('Amit', 32615),
 ('#FridayFeeling', 25321),
 ('Elise Stefanik', 25245),
 ('#fridaymorning', 24908),
 ('BRAD', 22833),
 ('Gen X', 16581),
 ('#FridayThoughts', 16213),
 ('Lock Screen', 16142),
 ('#MassEffectLegendaryEdition', 15377),
 ('HAPPY FRIDAY EVERYONE', 10160)]

In [83]:
import plotly.express as px
df = px.data.gapminder()
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


## Personal Tweets and Analysis

In [10]:
timeline = api.home_timeline()
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

Imran Khan said انسانیت کی تمام حدود اور عالمی قوانین پامال کرتے ہوئے خصوصاً رمضان المبارک کے دوران اسرائیلی فوجیوں کے قبلہ اول یعن… https://t.co/1KzxeLmzI2
Imran Khan said Strongly condemn Israeli Forces' attack esp during Ramazan on Palestinians in Qibla-e-Awaal, Al-Aqsa Mosque, violat… https://t.co/UpMZWC1iuP
SpaceX said Deployment of 60 Starlink satellites confirmed https://t.co/RarbrcfUml
Narendra Modi said Saddened by the demise of INA veteran Lalti Ram Ji. His courage and contributions to India’s freedom struggle will… https://t.co/E5hKmAdkXW
SpaceX said Falcon 9’s first stage has landed on the Just Read the Instructions droneship, completing this booster’s 10th launc… https://t.co/48pJZyHq1i
SpaceX said Liftoff! https://t.co/MZ4ct2Spnj
SpaceX said Watch the first tenth flight of an orbital class rocket booster → https://t.co/bJFjLCzWdK  https://t.co/MDM6RNlN4J
enzo said RT @Abz4ktt: I beg (if it is Laylatul Qadr tonight Inshallah) all of us Muslimeen in the world rn to all rais

# Create an NLP from twitter API

In [19]:
tweets = tweepy.Cursor(api.search, q='Drake', result_type='popular').items(1)
tweet_list = []
neg = []
neu = []
pos = []
for tweet in tweets:
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']

{'neg': 0.0, 'neu': 0.894, 'pos': 0.106, 'compound': 0.3612}
